## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [3]:
import os
from pathlib import Path
import sys
sys.path.append("C:/Users/EISERK01/eeg-projects/Code/eeg_toolkit")
from eeg_toolkit import utils, preprocess
from eeg_toolkit import source_localization as sl
import mne

0 files missing from root.txt in C:\Users\EISERK01\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\EISERK01\mne_data\MNE-fsaverage-data\fsaverage


### Paths

In [5]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


In [6]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [7]:
# viable subjects
sub_ids = ['018', '020', '021', '022', '023', '024', '027', '029','031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '052', '056', '057', '059', 'C2.', 'C3.', 'C6.', 'C7.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C24', 'C25', 'C26', 'C27']  
    
print(sub_ids)

['023', '029', '034', '041', 'C2.', 'C3.', 'C6.', 'C7.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C24', 'C25', 'C26', 'C27']


In [8]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'   
]
 

# CP Subs with widespread pain
sub_ids_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

 
sub_ids = sub_ids_CP + sub_ids_HC + sub_ids_WSP + sub_ids_LP

# keep only unique
sub_ids = list(set(sub_ids))

In [9]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 31
Controls: 19
Total: 50


#### Look for subjects who do not have EO or EC cropped data

In [12]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
yes_eyes_open = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-epochs.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open epochs data")
    else:
        yes_eyes_open.append(sub_id)
        
print(no_eyes_open)
print(yes_eyes_open)

[]
['C14', 'C27', 'C15', '020', '044', '036', '042', '029', '023', 'C10', '059', 'C9.', '039', '022', '034', 'C6.', '037', '041', '046', '024', '032', '033', 'C17', 'C24', '052', '056', '040', 'C16', '018', 'C13', '050', '031', 'C7.', '048', 'C19', '057', 'C3.', '043', 'C2.', '049', 'C18', 'C26', 'C11', '035', '038', '045', 'C12', '021', 'C25', '027']


In [13]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh', # Right Medial-OFC
]

### Compute STCs

In [15]:
nan_subjects=[]
zscored_epochs_save_path = None
EC_resting_save_path = None
EO_resting_save_path = None

methods = [
    'MNE',
    # 'dSPM',
    ]

for sub_id in sub_ids: 
    
    for save_stc_mat in [
        False,
        # True,
        ]:    
        for method in methods:
            print(f"Source Localization Method: {method}")                          

            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method})')
                           
            EO_epochs_save_path = stc_path / "Eyes Open"
            save_paths = [EO_epochs_save_path] 
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = True
                EO_bool = False # TODO: re-compute for just epochs with noise_cov instead of 
                # just noise var

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = sl.to_source(sub_id,
                                                             processed_data_path,
                                                             zscored_epochs_save_path,
                                                             EC_resting_save_path,
                                                             EO_resting_save_path,
                                                             EO_epochs_save_path,
                                                             roi_names,
                                                             times_tup,
                                                             method=method,
                                                             return_zepochs = False,
                                                             return_EC_resting = False,
                                                             return_EO_resting = False,
                                                             return_EO_epochs = True,
                                                             average_dipoles=average_dipoles,
                                                             save_stc_mat=save_stc_mat,
                                                             save_inv=save_inv,)
            utils.clear_display()
            
            # add nan subjects to list if not None
            if sub_id_if_nan is not None:
                nan_subjects.append(sub_id_if_nan)

Source Localization Method: MNE
Using up to 1558 segments
Number of samples used : 124640
[done]
Computing rank from covariance with rank=None
    Using tolerance 1.1e-13 (2.2e-16 eps * 64 dim * 7.7  max singular value)
    Estimated rank (eeg): 61
    EEG: rank 61 computed from 64 data channels with 0 projectors
0 projection items activated
    EEG regularization : 0.1
Computing rank from covariance with rank={'eeg': 61}
    Setting small EEG eigenvalues to zero (without PCA)


C:\Users/EISERK01/eeg-projects/Code/eeg_toolkit\eeg_toolkit\source_localization.py:342: RuntimeWarning: This filename (C:/Users/EISERK01/eeg-projects/Chronic Low Back Pain Study/Data/Processed Data/027_eyes_open-epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  raw_eo_epochs = mne.read_epochs(raw_eo_epochs_path)
C:\Users/EISERK01/eeg-projects/Code/eeg_toolkit\eeg_toolkit\source_localization.py:342: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  raw_eo_epochs = mne.read_epochs(raw_eo_epochs_path)
C:\Users/EISERK01/eeg-projects/Code/eeg_toolkit\eeg_toolkit\source_localization.py:342: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  raw_eo_epochs = mne.read_epochs(raw_eo_epochs_path)
C:\Users/EISERK01/eeg-projects/Code/eeg_toolkit\eeg_toolkit\source_localization.py:343: RuntimeWarning: An average reference pro